# Challenge 12/12/2024

Challenge instructions [here](https://adventofcode.com/2024/day/12)

## Highlights & Notes

- **Goal**: Find the total cost of fencing the garden.
- The total cost is the sum f the cost of each region
- Each region cost is the multiplication of its perimeter and area
- The area is the number of symbols in the region
- The perimeter is the number of sides of the region

Resolution:

- Find all independent region


## Setup & Imports


In [5]:
import numpy as np
import sys
import os

# Get utility functions
sys.path.append(os.path.abspath('../utils'))
from utils import read_input_file, remove_newline_char, split_lines

# Quick ANSI color code shortcuts
r = "\033[31m";y = "\033[33m";g = "\033[32m";b = "\033[34m";e = "\033[0m"

## Part 1


In [6]:
from collections import deque

def dfs(grid, visited, x, y, symbol):
    # Base case: If out of bounds or already visited or not the same symbol
    if (x < 0 or x >= len(grid) or
        y < 0 or y >= len(grid[0]) or
        visited[x][y] or grid[x][y] != symbol):
        return 0, 0  # Area, Perimeter

    visited[x][y] = True  # Mark the cell as visited
    area = 1  # Current cell counts as area
    perimeter = 0  # Initialize perimeter for this cell

    # Directions: Up, Down, Left, Right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dx, dy in directions:
        nx, ny = x + dx, y + dy

        if (0 <= nx < len(grid) and 0 <= ny < len(grid[0])):
            if grid[nx][ny] == symbol:
                if not visited[nx][ny]:
                    # Recurse into the neighbor cell
                    a, p = dfs(grid, visited, nx, ny, symbol)
                    area += a
                    perimeter += p
            else:
                # Neighboring cell is different, so this edge contributes to perimeter
                perimeter += 1
        else:
            # Edge of the grid contributes to perimeter
            perimeter += 1

    return area, perimeter


def bfs(grid, visited, x, y, symbol):
    queue = deque()
    queue.append((x, y))
    visited[x][y] = True  # Mark the starting cell as visited
    area = 1
    perimeter = 0

    while queue:
        cx, cy = queue.popleft()
        
        # Number of sides contributing to the perimeter from this cell
        cellPerimeter = 0

        # Directions: Up, Down, Left, Right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for dx, dy in directions:
            nx, ny = cx + dx, cy + dy

            if (0 <= nx < len(grid) and 0 <= ny < len(grid[0])):
                if grid[nx][ny] == symbol:
                    if not visited[nx][ny]:
                        visited[nx][ny] = True
                        queue.append((nx, ny))
                        area += 1
                else:
                    # Neighboring cell is different
                    cellPerimeter += 1
            else:
                # Edge of the grid
                cellPerimeter += 1
        perimeter += cellPerimeter

    return area, perimeter

In [7]:
%%time
# Sample grid
grid = remove_newline_char(read_input_file("input.txt"))

rows = len(grid)
cols = len(grid[0])
visited = [[False for _ in range(cols)] for _ in range(rows)]
regions = []

for i in range(rows):
    for j in range(cols):
        if not visited[i][j]:
            symbol = grid[i][j]
            area, perimeter = bfs(grid, visited, i, j, symbol)
            regions.append({'symbol': symbol, 'area': area, 'perimeter': perimeter})

# Calculate the total cost
totalCost = sum(region['area'] * region['perimeter'] for region in regions)
print(f"Total cost: {totalCost}")

# BFS ~ 40ms, DFS ~ 35ms

Total cost: 1473620
CPU times: total: 31.2 ms
Wall time: 59.4 ms


## Part 2

Instructions [here](https://adventofcode.com/2024/day/2#part2)

Counting sides is equivalent to counting corners, thanks @BillyJay


In [23]:
def direction_change(currentDir, nextDir):
    return currentDir is not None and currentDir != nextDir

def print_around_cell(grid, x, y):
    # Check we are not out of bounds
    if x-1 < 0 or x+2 >= len(grid) or y-1 < 0 or y+2 >= len(grid[0]):
        return

    for i in range(x - 1, x + 2):
        for j in range(y - 1, y + 2):
            print(grid[i][j], end=" ")
        print()
    print()

def bfs_corners(grid, visited, x, y, symbol):
    queue = deque()
    queue.append((x, y))
    visited[x][y] = True  # Mark the starting cell as visited
    area = 1
    nbCorners = 0

    # Directions: Up, Down, Left, Right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    currentDir = None

    while queue:
        cx, cy = queue.popleft()

        for newDir, (dx, dy) in enumerate(directions):
            nx, ny = cx + dx, cy + dy

            if (0 <= nx < len(grid) and 0 <= ny < len(grid[0])):
                if grid[nx][ny] == symbol:
                    if not visited[nx][ny]:
                        visited[nx][ny] = True
                        queue.append((nx, ny))
                        area += 1
                else:
                    # Neighboring cell is different
                    if direction_change(currentDir, newDir):
                        # print_around_cell(grid, nx, ny)
                        nbCorners += 1
                    currentDir = newDir
            else:
                # Edge of the grid
                if direction_change(currentDir, newDir):
                    # print_around_cell(grid, nx, ny)
                    nbCorners += 1 
                currentDir = newDir

    return area, nbCorners

In [26]:
%%time
# Sample grid
grid = remove_newline_char(read_input_file("demo.txt"))

rows = len(grid)
cols = len(grid[0])
visited = [[False for _ in range(cols)] for _ in range(rows)]
regions = []

for i in range(rows):
    for j in range(cols):
        if not visited[i][j]:
            symbol = grid[i][j]
            area, corners = bfs_corners(grid, visited, i, j, symbol)
            regions.append({'symbol': symbol, 'area': area, 'corners': corners})

# Calculate the total cost
totalCost = sum(region['area'] * region['corners'] for region in regions)
print(f"Total cost: {totalCost}")

Total cost: 1696
CPU times: total: 0 ns
Wall time: 2.14 ms
